# SVD

## Import libraries

In [6]:
from lib.models import RecommendSystemModel

from typing import List, Any, Tuple,Union
from numpy.typing import NDArray
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## update class 
https://stackoverflow.com/questions/45161393/jupyter-split-classes-in-multiple-cells

In [2]:
import functools
def update_class(
    main_class=None, exclude=("__module__", "__name__", "__dict__", "__weakref__")
):
    """Class decorator. Adds all methods and members from the wrapped class to main_class

    Args:
    - main_class: class to which to append members. Defaults to the class with the same name as the wrapped class
    - exclude: black-list of members which should not be copied
    """

    def decorates(main_class, exclude, appended_class):
        if main_class is None:
            main_class = globals()[appended_class.__name__]
        for k, v in appended_class.__dict__.items():
            if k not in exclude:
                setattr(main_class, k, v)
        return main_class

    return functools.partial(decorates, main_class, exclude)

In [3]:
class MyClass:
    def method1(self):
        print("method1")
me = MyClass()


In [4]:
@update_class()
class MyClass:
    def method2(self):
        print("method2")
me.method1()
me.method2()

method1
method2


## The ML model

In [3]:
class SVDModel(RecommendSystemModel):
    def __init__(self) -> None:
        self.data:pd.DataFrame 
        self.train:pd.DataFrame
        self.valid:pd.DataFrame
        super().__init__()

In [4]:
@update_class()
class SVDModel(RecommendSystemModel):
    def split(self,ratio: float, tensor: bool = False) -> List[NDArray]:
        self.train = np.zeros((len(self.data), len(self.data[0]))).tolist()
        self.valid = np.zeros((len(self.data), len(self.data[0]))).tolist()

        for i in range(len(self.data)):
            for j in range(len(self.data[i])):
                if self.data[i][j] > 0:
                    if np.random.binomial(1, ratio, 1):
                        self.train[i][j] = self.data[i][j]
                    else:
                        self.valid[i][j] = self.data[i][j]


In [8]:
@update_class()
class SVDModel(RecommendSystemModel):
    def data_loader(self, path:str, nrows:int=None, skiprows=None) -> None:
        print("Dataloader")
        self.data = pd.read_csv(path,low_memory=False,nrows=nrows,skiprows=skiprows)
        # return super().data_loader()

In [8]:
@update_class()
class SVDModel(RecommendSystemModel):
    def learn_to_recommend(self,features: int = 10, lr: float = 0.0002, epochs: int = 101, weight_decay: float = 0.02, stopping: float = 0.001) -> Tuple[NDArray, NDArray, float, float]:
        print(789)
        # return super().learn_to_recommend(data, features, lr, epochs, weight_decay, stopping)

In [9]:
@update_class()
class SVDModel(RecommendSystemModel):
    def prediction(P: NDArray, Q: NDArray, u: int, i: int) -> float:
        print(321)
        # return super().prediction(P, Q, u, i)

In [10]:
@update_class()
class SVDModel(RecommendSystemModel):
    def loss(data: Any, P: NDArray, Q: NDArray) -> float:
        print(654)
        # return super().loss(data, P, Q)

In [22]:
@update_class()
class SVDModel(RecommendSystemModel):
    def svd(self):
        print(987)
        # return super().svd()

In [23]:
svd = SVDModel()

svd.svd()
svd.learn_to_recommend(2)

987
789
